<a href="https://colab.research.google.com/github/Theonimfi/Bert-based-preprocessing-techniques-acceleration/blob/main/BERT_preprocessing_acceleration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow==1.15.0

  Using cached tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl (412.3 MB)
  Using cached tensorflow_estimator-1.15.1-py2.py3-none-any.whl (503 kB)
  Using cached tensorboard-1.15.0-py3-none-any.whl (3.8 MB)
  Using cached gast-0.2.2-py3-none-any.whl
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Successfully uninstalled gast-0.4.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.6.0
    Uninstalling tensorflow-2.6.0:
      Successfully uninstalled tensorflow-2.6.0
ERROR: pip's dependency resolver does not currently 

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from sklearn.model_selection import train_test_split
import os

print("tensorflow version : ", tf.__version__)
print("tensorflow_hub version : ", hub.__version__)

tensorflow version :  1.15.0
tensorflow_hub version :  0.12.0


In [ ]:
!pip install bert-tensorflow==1.0.1

In [ ]:
#Importing BERT modules
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [ ]:
# Load the dataset.
!pip install numpy
!pip install pandas
!pip install nltk
!pip install -q -U tensorflow-text
!pip install -q tf-models-official
!pip install contractions
!pip install textblob

import numpy as np
import pandas as pd
import nltk

In [ ]:
# https://www.codegrepper.com/code-examples/python/how+to+read+csv+file+from+google+drive+on+google+colab+
#Derive the id from the google drive shareable link.
#For the file at hand the link is as below
URL = 'https://drive.google.com/file/d/10j1dwpPCk7XAHCTX7gczX2woXzMZ5Dva/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]
#df = pd.read_pickle(path)
df = pd.read_csv(path)
# df = df.head(100)
df = df[['text','airline_sentiment']]

In [ ]:
from nltk.stem.porter import PorterStemmer
from nltk import word_tokenize
import string
import re
from datetime import datetime
from textblob import TextBlob
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
import contractions

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

LOWERCASE = False
REMOVE_NUMBERS = False
REMOVE_HASHTAGS = False
REMOVE_MENTIONS = False
REMOVE_PUNCTUATIONS = False
STEMMING = False

REMOVE_URLS = False
REPLACE_REPEAT_PUNCTUATION = False # repeat punctuation show intense emotion show we replace the punctuation with a word to capture the intensity of the text
HANDLE_CAPITALIZED_WORDS = False # capitalize words show intense emotion show we add a word before each capitalized word to capture the intensity of the text
EMOJIS = False # replace emojis with a word which indicates whether it is a positive or negative emoji (eg :) -> POS_EMOJI, :(( -> NEG_EMOJI)
SPELLING_CORRECTION = False # very computationally expensive maybe impossible to use it and we should just delete it

# choose which tech we use
STOP_WORDS_REMOVAL = False
REPEATED_CHARACTERS_REMOVAL = False
LEMMATIZATION = False
EXTRA_SPACE_REMOVAL = False
CONTRACTED_WORDS_EXPANDING = False
print(f'Beginning Pre-processing!')
current_time = datetime.now()



porter = PorterStemmer()

df['processed_text'] = df['text'].fillna('').apply(str)
if REPLACE_REPEAT_PUNCTUATION:
  df['processed_text'] = df['processed_text'].apply(lambda x: re.sub('\!\!+', 'multiExclamationMark ', x))
  df['processed_text'] = df['processed_text'].apply(lambda x: re.sub('\?\?+', 'multiQuestionMark ', x))



if LOWERCASE:
  df['processed_text'] = df['processed_text'].apply(lambda x: x.lower())

if REMOVE_MENTIONS:
  df['processed_text'] = df['processed_text'].apply(lambda x: re.sub("@[A-Za-z0-9_]+","", x))

if REMOVE_HASHTAGS:
  df['processed_text'] = df['processed_text'].apply(lambda x: re.sub("#[A-Za-z0-9_]+","", x))

if REMOVE_URLS:
  df['processed_text'] = df['processed_text'].apply(lambda x: re.sub(r'http\S+', '', x))

if REMOVE_NUMBERS:
  df['processed_text'] = df['processed_text'].apply(lambda x: re.sub(r'[0-9]+', '', x))

if REMOVE_PUNCTUATIONS:
  df['processed_text'] = df['processed_text'].apply(lambda x: "".join([char for char in x if char not in string.punctuation]))

if HANDLE_CAPITALIZED_WORDS:
  df['processed_text'] = df['processed_text'].apply(lambda x: " ".join(["ALL_CAPS " + word if (word.isupper() and len(word) > 2) else word for word in word_tokenize(x)]))

if SPELLING_CORRECTION:
  df['processed_text'] = df['processed_text'].apply(lambda x: " ".join([str(TextBlob(word).correct()) for word in word_tokenize(x)]))

if EMOJIS:

  def find_emojis(text, neg_emoticons, pos_emoticons, neg_emojis, pos_emojis):
      words = word_tokenize(text)
      for i, _ in enumerate(words):
        if words[i] in neg_emojis or words[i] in neg_emoticons:
          words[i] = "NEG_EMOJI"
        elif words[i] in pos_emojis or words[i] in pos_emoticons:
          words[i] = "POS_EMOJI"
      
      return " ".join(words)
    




  # files from here: https://github.com/modarwish1/sentimentr
  URL = 'https://drive.google.com/file/d/1IBTu3IgYECIl1SOdbPr4o8LUM1_avAas/view?usp=sharing'
  path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]
  neg_emoticons = pd.read_csv(path, sep="\t", header=None, usecols=[0])
  neg_emoticons.columns = ["emoticon"]
  neg_emoticons = neg_emoticons["emoticon"].to_list()
  # print(neg_emoticons)

  URL = 'https://drive.google.com/file/d/148G4WZablFzVQSmEak6DqABarn4EY2do/view?usp=sharing'
  path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]
  pos_emoticons = pd.read_csv(path, sep="\t", header=None, usecols=[0])
  pos_emoticons.columns = ["emoticon"]
  pos_emoticons = pos_emoticons["emoticon"].to_list()
  # print(pos_emoticons)

  URL = 'https://drive.google.com/file/d/1FTKcH1ackudDaeDi-ZveA9eIUYkJhsRK/view?usp=sharing'
  path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]
  neg_emojis = pd.read_csv(path, sep="\t", header=None, usecols=[0])
  neg_emojis.columns = ["emoticon"]
  neg_emojis = neg_emojis["emoticon"].to_list()
  # print(neg_emojis)

  URL = 'https://drive.google.com/file/d/1QOLDjOjYtkCBWsGwjfJ4bBGGsDqfg3Ck/view?usp=sharing'
  path = 'https://drive.google.com/uc?export=download&id='+URL.split('/')[-2]
  pos_emojis = pd.read_csv(path, sep="\t", header=None, usecols=[0])
  pos_emojis.columns = ["emoticon"]
  pos_emojis = pos_emojis["emoticon"].to_list()
  # print(pos_emojis)
  df['processed_text'] = df['processed_text'].apply(lambda x: find_emojis(x, neg_emoticons, pos_emoticons, neg_emojis, pos_emojis))

if STEMMING:
  df['processed_text'] = df['processed_text'].apply(lambda x: " ".join([porter.stem(word) for word in word_tokenize(x)]))




column_0 = 'processed_text'
column_1 = 'airline_sentiment'
text_column = df[column_0]

# some prepard class and method
class RepeatReplacer(object):
  def __init__(self):
    self.regex = re.compile(r'(\w*)(\w)\2(\w*)')
    self.repl = r'\1\2\3'
  
  def replace(self, word):
    if wordnet.synsets(word):
      return word
    loop_res = self.regex.sub(self.repl, word)
    if(word == loop_res):
      return loop_res
    else:
      return self.replace(loop_res)
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None



# start processing
if STOP_WORDS_REMOVAL:
  stop_words = set(stopwords.words('english'))
  # processing the text in loop
  for t in text_column:
    new_t = ' '.join([word for word in t.lower().split(' ') if word not in stop_words])
    text_column.replace({t:new_t}, inplace=True)
if REPEATED_CHARACTERS_REMOVAL:
  replacer = RepeatReplacer()
  # processing the text in loop
  for t in text_column:
    word_list = []
    for word in t.lower().split(' '):
      word_list.append(replacer.replace(word))
    new_t = ' '.join(word_list)
    text_column.replace({t:new_t}, inplace=True)
if LEMMATIZATION:
  lemmatizer = WordNetLemmatizer()
  # processing the text in loop
  for t in text_column:
    tokens = word_tokenize(t)
    tagged_t = pos_tag(tokens)
    lemmas_t = []
    for tag in tagged_t:
      wordnet_pos = get_wordnet_pos(tag[1]) or wordnet.NOUN
      lemmas_t.append(lemmatizer.lemmatize(tag[0], pos=wordnet_pos)) 
    new_t = ' '.join(lemmas_t)
    text_column.replace({t:new_t}, inplace=True)
if EXTRA_SPACE_REMOVAL:
  # processing the text in loop
  for t in text_column:
    new_t = re.sub(' +', ' ', t)
    text_column.replace({t:new_t}, inplace=True)
if CONTRACTED_WORDS_EXPANDING:
# processing the text in loop
  for t in text_column:
    word_list = []
    for word in t.lower().split(' '):
      word_list.append(contractions.fix(word))
    new_t = ' '.join(word_list)
    text_column.replace({t:new_t}, inplace=True)


print("Pre-processing took time ", datetime.now() - current_time)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
print(df)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
Beginning Pre-processing!
Pre-processing took time  0:00:00.007581
                                                    text  ...                                     processed_text
0                    @VirginAmerica What @dhepburn said.  ...                @VirginAmerica What @dhepburn said.
1      @VirginAmerica plus you've added commercials t...  ...  @VirginAmerica plus you've added commercials t...
2      @VirginAmerica I didn't today... Must mean I n...  ...

In [ ]:
filtered_columns=['airline_sentiment','processed_text']
df= df.reindex(columns=filtered_columns)
df.head()


,airline_sentiment,processed_text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,@VirginAmerica it's really aggressive to blast...
4,negative,@VirginAmerica and it's a really big bad thing...


In [ ]:
cleanup_nums = {"airline_sentiment": {"positive": 1, "negative": 0,"neutral":2}}
df = df.replace(cleanup_nums)
df.head()

,airline_sentiment,processed_text
0,2,@VirginAmerica What @dhepburn said.
1,1,@VirginAmerica plus you've added commercials t...
2,2,@VirginAmerica I didn't today... Must mean I n...
3,0,@VirginAmerica it's really aggressive to blast...
4,0,@VirginAmerica and it's a really big bad thing...


In [ ]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(df, test_size=0.2)
train.head(5)

,airline_sentiment,processed_text
9025,2,"@USAirways ""Airport snow removal method #22......"
5722,0,@SouthwestAir the sign in screen had major mob...
12676,0,@AmericanAir Pay for my accommodations when yo...
13042,1,@AmericanAir Thanks for asking On second plane...
7983,0,@JetBlue Jan 5? That's not summer http://t.co/...


In [ ]:
DATA_COLUMN = 'processed_text'
LABEL_COLUMN = 'airline_sentiment'
# The list containing all the classes (train['SECTION'].unique())
label_list = [0, 1, 2]

In [ ]:

train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

val_InputExamples = val.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

In [ ]:

train_InputExamples

9025     <bert.run_classifier.InputExample object at 0x...
5722     <bert.run_classifier.InputExample object at 0x...
12676    <bert.run_classifier.InputExample object at 0x...
13042    <bert.run_classifier.InputExample object at 0x...
7983     <bert.run_classifier.InputExample object at 0x...
                               ...                        
7810     <bert.run_classifier.InputExample object at 0x...
6385     <bert.run_classifier.InputExample object at 0x...
8438     <bert.run_classifier.InputExample object at 0x...
10102    <bert.run_classifier.InputExample object at 0x...
339      <bert.run_classifier.InputExample object at 0x...
Length: 11712, dtype: object

In [ ]:
print("Row 0 - guid of training set : ", train_InputExamples.iloc[0].guid)
print("\n__________\nRow 0 - text_a of training set : ", train_InputExamples.iloc[0].text_a)
print("\n__________\nRow 0 - text_b of training set : ", train_InputExamples.iloc[0].text_b)
print("\n__________\nRow 0 - label of training set : ", train_InputExamples.iloc[0].label)

Row 0 - guid of training set :  None

__________
Row 0 - text_a of training set :  @USAirways "Airport snow removal method #22..."
Keep up the good work folks, this is where Cessna's become 747's! http://t.co/0JJT4X3YxG

__________
Row 0 - text_b of training set :  None

__________
Row 0 - label of training set :  2


In [ ]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
#!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [ ]:
print(tokenizer.tokenize(train_InputExamples.iloc[0].text_a))

['@', 'usa', '##ir', '##ways', '"', 'airport', 'snow', 'removal', 'method', '#', '22', '.', '.', '.', '"', 'keep', 'up', 'the', 'good', 'work', 'folks', ',', 'this', 'is', 'where', 'cessna', "'", 's', 'become', '747', "'", 's', '!', 'http', ':', '/', '/', 't', '.', 'co', '/', '0', '##j', '##j', '##t', '##4', '##x', '##3', '##yx', '##g']


In [ ]:

# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

# Convert our train and validation features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 11712


INFO:tensorflow:Writing example 0 of 11712


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: None


...

INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0


INFO:tensorflow:label: 2 (id = 2)


INFO:tensorflow:label: 2 (id = 2)


In [ ]:
print("Sentence : ", train_InputExamples.iloc[0].text_a)
print("-"*30)
print("Tokens : ", tokenizer.tokenize(train_InputExamples.iloc[0].text_a))
print("-"*30)
print("Input IDs : ", train_features[0].input_ids)
print("-"*30)
print("Input Masks : ", train_features[0].input_mask)
print("-"*30)
print("Segment IDs : ", train_features[0].segment_ids)

Sentence :  @USAirways "Airport snow removal method #22..."
Keep up the good work folks, this is where Cessna's become 747's! http://t.co/0JJT4X3YxG
------------------------------
Tokens :  ['@', 'usa', '##ir', '##ways', '"', 'airport', 'snow', 'removal', 'method', '#', '22', '.', '.', '.', '"', 'keep', 'up', 'the', 'good', 'work', 'folks', ',', 'this', 'is', 'where', 'cessna', "'", 's', 'become', '747', "'", 's', '!', 'http', ':', '/', '/', 't', '.', 'co', '/', '0', '##j', '##j', '##t', '##4', '##x', '##3', '##yx', '##g']
------------------------------
Input IDs :  [101, 1030, 3915, 4313, 14035, 1000, 3199, 4586, 8208, 4118, 1001, 2570, 1012, 1012, 1012, 1000, 2562, 2039, 1996, 2204, 2147, 12455, 1010, 2023, 2003, 2073, 27664, 1005, 1055, 2468, 25374, 1005, 1055, 999, 8299, 1024, 1013, 1013, 1056, 1012, 2522, 1013, 1014, 3501, 3501, 2102, 2549, 2595, 2509, 17275, 2290, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  
  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]
  print(output_layer.shape)

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)
    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

In [ ]:
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        
        return {
            "eval_accuracy": accuracy,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
            }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)
      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

In [ ]:
OUTPUT_DIR = '/content/drive/MyDrive/DataMining_NLP'

In [ ]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where the learning rate is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 300
SAVE_SUMMARY_STEPS = 100

# Compute train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [ ]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/MyDrive/DataMining_NLP', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc7a10e2e10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/content/drive/MyDrive/DataMining_NLP', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 300, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fc7a10e2e10>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:

# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

# Create an input function for validating. drop_remainder = True for using TPUs.
val_input_fn = run_classifier.input_fn_builder(
    features=val_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [ ]:
#Training the model
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

Instructions for updating:
Use standard file APIs to delete files with this prefix.


Instructions for updating:
Use standard file APIs to delete files with this prefix.


INFO:tensorflow:Loss for final step: 0.015623061.


INFO:tensorflow:Loss for final step: 0.015623061.


Training took time  0:31:36.552985


In [ ]:
estimator.evaluate(input_fn=val_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


(?, 768)


...

INFO:tensorflow:Saving dict for global step 1098: eval_accuracy = 0.84118855, false_negatives = 197.0, false_positives = 142.0, global_step = 1098, loss = 0.59648144, true_negatives = 1673.0, true_positives = 916.0


INFO:tensorflow:Saving dict for global step 1098: eval_accuracy = 0.84118855, false_negatives = 197.0, false_positives = 142.0, global_step = 1098, loss = 0.59648144, true_negatives = 1673.0, true_positives = 916.0


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1098: /content/drive/MyDrive/DataMining_NLP/model.ckpt-1098


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1098: /content/drive/MyDrive/DataMining_NLP/model.ckpt-1098


{'eval_accuracy': 0.84118855,
 'false_negatives': 197.0,
 'false_positives': 142.0,
 'global_step': 1098,
 'loss': 0.59648144,
 'true_negatives': 1673.0,
 'true_positives': 916.0}